# Specific Heat Capacity from pycalphad

In this notebook, the interaction between PyRolL and pycalphad is demonstrated. 
In this example, the calphad method is used to calculate the specific heat capacity of the low-carbon steel C15 (1.0401) using free `tdb` databases provided by [MatCalc Engineering GmbH](https://www.matcalc.at/index.php/databases/open-databases). The measured chemical composition (wt%) of a example can be seen in the following table: 

| Steel grade  | C    | Si   | Mn   | P     | S      | Fe    |
|--------------|------|------|------|-------|--------|-------|
| C15 (1.0401) | 0.16 | 0.16 | 0.02 | 0.002 | 0.0065 | 98.43 |

Further, thermodynamical database files (`tdb`) for other alloying systems can be found [here](https://avdwgroup.engin.brown.edu/). The specific-heat-capacity is mostly important when considering cooling processes which often follow a hot rolling process. As always, we first import the necessary python modules.

In [16]:
import numpy as np
import pyroll.core as pr
import matplotlib.pyplot as plt
import pycalphad.variables as v

from pycalphad import equilibrium
from pycalphad import Database, Model

## Conversion of weight-fractions to molar-fractions
The industrial standard, how to provide the chemical composition of an alloy is to use weight-fractions. In the scientific realm, however, molar-fractions are more common. Therefore, we need a function which converts weight-fractions to molar fractions. Further, we need a function which calculates the total molar mass of the alloy. These values are needed, since the unit of the specifc-heat-capacity returned by pycalphad has to be converted. $$ {c_P [{J \over kg K}]}  = {c_P [{J \over mol K}] \over m_{molar}[{kg \over mol}]}  $$

In [17]:
 molar_masses_all_elements_gramm_per_mole = {
    'H': 1.008, 'He': 4.0026, 'Li': 6.94, 'Be': 9.0122, 'B': 10.81,
    'C': 12.011, 'N': 14.007, 'O': 15.999, 'F': 18.998, 'Ne': 20.180,
    'Na': 22.990, 'Mg': 24.305, 'Al': 26.982, 'Si': 28.085, 'P': 30.974,
    'S': 32.06, 'Cl': 35.45, 'Ar': 39.948, 'K': 39.098, 'Ca': 40.078,
    'Sc': 44.956, 'Ti': 47.867, 'V': 50.942, 'Cr': 51.996, 'Mn': 54.938,
    'Fe': 55.845, 'Co': 58.933, 'Ni': 58.693, 'Cu': 63.546, 'Zn': 65.38,
    'Ga': 69.723, 'Ge': 72.63, 'As': 74.922, 'Se': 78.971, 'Br': 79.904,
    'Kr': 83.798, 'Rb': 85.468, 'Sr': 87.62, 'Y': 88.906, 'Zr': 91.224,
    'Nb': 92.906, 'Mo': 95.95, 'Tc': 98.0, 'Ru': 101.07, 'Rh': 102.91,
    'Pd': 106.42, 'Ag': 107.87, 'Cd': 112.41, 'In': 114.82, 'Sn': 118.71,
    'Sb': 121.76, 'Te': 127.6, 'I': 126.9, 'Xe': 131.29, 'Cs': 132.91,
    'Ba': 137.33, 'La': 138.91, 'Ce': 140.12, 'Pr': 140.91, 'Nd': 144.24,
    'Pm': 145.0, 'Sm': 150.36, 'Eu': 151.96, 'Gd': 157.25, 'Tb': 158.93,
    'Dy': 162.5, 'Ho': 164.93, 'Er': 167.26, 'Tm': 168.93, 'Yb': 173.05,
    'Lu': 174.97, 'Hf': 178.49, 'Ta': 180.95, 'W': 183.84, 'Re': 186.21,
    'Os': 190.23, 'Ir': 192.22, 'Pt': 195.08, 'Au': 196.97, 'Hg': 200.59,
    'Tl': 204.38, 'Pb': 207.2, 'Bi': 208.98, 'Po': 209.0, 'At': 210.0,
    'Rn': 222.0, 'Fr': 223.0, 'Ra': 226.0, 'Ac': 227.0, 'Th': 232.04,
    'Pa': 231.04, 'U': 238.03, 'Np': 237.0, 'Pu': 244.0, 'Am': 243.0,
    'Cm': 247.0, 'Bk': 247.0, 'Cf': 251.0, 'Es': 252.0, 'Fm': 257.0,
    'Md': 258.0, 'No': 259.0, 'Lr': 266.0
}


def convert_mass_fractions_to_molar_fractions(mass_fractions: dict[str, float]) -> dict[str, float]:
    total_mass_of_alloy = sum(mass_fractions[element] for element in mass_fractions)
    molar_fractions = {}

    for element in mass_fractions:
        try:
            molar_fractions[element] = mass_fractions[element] / (
                    molar_masses_all_elements_gramm_per_mole[element] * total_mass_of_alloy)
        except KeyError:
            print(f"Error: Molar mass not found for element {element}")

    return molar_fractions


def molar_mass(mass_fractions: dict[str, float]) -> float:
    return (sum(
        (mass_fractions[element] / 100.0) * molar_masses_all_elements_gramm_per_mole[element] for element in
        mass_fractions)) / 1000


## Definition of incoming profile
Now let's define a profile and use it to calculate the molar fractions as well as the molar mass.

In [18]:
profile = pr.Profile.round(
    diameter=50e-3,
    temperature=1200 + 273.15,
    material=["steel", "C15"],
    chemical_composition={
        "C": 0.16,
        "Si": 0.16,
        "Mn": 0.02,
        "P": 0.002,
        "S": 0.0065,
        "Fe": 98.43
    },
    density=7.5e3
)

In [24]:
molar_fractions = convert_mass_fractions_to_molar_fractions(profile.chemical_composition)
total_molar_mass = molar_mass(profile.chemical_composition)

molar_fractions, total_molar_mass

({'C': 0.00013485851986570112,
  'Si': 5.7674405629586476e-05,
  'Mn': 3.68548564314986e-06,
  'P': 6.536876420977821e-07,
  'S': 2.0525200666124226e-06,
  'Fe': 0.017843529488963394},
 0.05504607808)

## Stepped Equilibrium 
Now we have to calculate a stepped equilibrium using the `pycalphad` package. This equilibrium gives us information about the phase compositions of the alloy at various temperatures between a start and a stop temperature. Further information we have to provide to run the calculations is the atmospheric pressure as well as the number of moles we want to consider. Also, we need information about the expected phases. This information can be gained from literature or a phase diagram. In this case we expect the liquid phase (LIQUID), cementite (CEMENTITE), austenite (FCC_A1) as well as ferrite (BCC_A2) regarding the temperature. At last the composition (chemical elements) of the alloy.

In [ ]:
start_temperature_kelvin = 298.15 + 273.15
end_temperature_kelvin = 1700 + 273.15
number_of_moles = 1
atmospheric_pressure = 101325

conditions = {
    v.X('C'): molar_fractions['C'],
    v.X('SI'): molar_fractions['Si'],
    v.X('MN'): molar_fractions['Mn'],
    v.X('P'): molar_fractions['P'],
    v.X('S'): molar_fractions['S'],
    v.N: number_of_moles,
    v.P: atmospheric_pressure,
    v.T: (start_temperature_kelvin, end_temperature_kelvin, 10)  # temperature (start, stop, step)
}
db = Database('mc_fe_v2.059.pycalphad.tdb')
phases = ['LIQUID', 'FCC_A1', 'BCC_A2', 'CEMENTITE']
composition = ['FE', 'C', 'CU', 'MN', 'P', 'S', 'SI', 'VA']
eq = equilibrium(db, composition, phases, conditions, output=['heat_capacity', 'degree_of_ordering'])